# LANGCHAIN & CASSANDRA DB

### RAG-USING-OPENAI

In [3]:
pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00


In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [10]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.indexes.vectorstore import VectorStoreIndexWrapper #wrap all the embeddings in one package to enhance retrival.
from datasets import load_dataset # can load any kind of dataset.
import cassio #to get connected with astra db.

In [11]:
! pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [12]:
from PyPDF2 import PdfReader #To read the PDF documents

In [13]:
#API-Keys
ASTRA_DB_APPLICATION_TOKEN='AstraCS:ucBeZgLYzdCFZquSxsYzPKfk:6858282b28b4f3860ca1521e9f3f82919a228a2f78205d5959739046ca80c534'
ASTRA_DB_ID= '59f48845-f00f-4f86-be09-4cd97d2dd4d0'
OPENAI_API_KEY= #

In [16]:
pdf_reader=PdfReader('/content/drive/MyDrive/RAG/attention-is-all-you-need.pdf')

In [17]:
from typing_extensions import Concatenate
text=''
for i,page in enumerate(pdf_reader.pages):
  content=page.extract_text()
  if content:
    text+=content

In [20]:
print(text[:500])

Attention Is All You Need
Ashish Vaswani
Google Brain
avaswani@google.comNoam Shazeer
Google Brain
noam@google.comNiki Parmar
Google Research
nikip@google.comJakob Uszkoreit
Google Research
usz@google.com
Llion Jones
Google Research
llion@google.comAidan N. Gomezy
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser
Google Brain
lukaszkaiser@google.com
Illia Polosukhinz
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
conv


In [25]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(132533130952912) 59f48845-f00f-4f86-be09-4cd97d2dd4d0-us-east1.db.astra.datastax.com:29042:5cc593a6-6bcd-41ec-851b-18d04c8e15b3> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [26]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0.1)
embeddings= OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [28]:
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="qa_mini_db",
    session=None,
    keyspace=None
)

In [29]:
from langchain_text_splitters import CharacterTextSplitter
# We need to split the text so that it can't exceed the token size.
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
splited_Text=text_splitter.split_text(text)

In [30]:
splited_Text[:20]

['Attention Is All You Need\nAshish Vaswani\x03\nGoogle Brain\navaswani@google.comNoam Shazeer\x03\nGoogle Brain\nnoam@google.comNiki Parmar\x03\nGoogle Research\nnikip@google.comJakob Uszkoreit\x03\nGoogle Research\nusz@google.com\nLlion Jones\x03\nGoogle Research\nllion@google.comAidan N. Gomez\x03y\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser\x03\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin\x03z\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,',
 'performing models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and co

In [31]:
#Will create the Embeddings itself and store it into the Database.
astra_vector_store.add_texts(splited_Text)
print("Inserted %i PDFLines " %len(splited_Text))

Inserted 55 PDFLines 


### Created the Indexes for fast and Generic Retrivals

In [36]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store) # to enhance the retrival and indexes are created for the each embeddings.

### Question from the Document

In [39]:
query_question=input("Enter your 1st Question: ").strip()
print("\n\n QUESTION %s" %query_question)
answer=astra_vector_index.query(query_question,llm=llm).strip()
print("\n\n ANSWER: %s" %answer)

Enter your 1st Question: What is the main feature of the Transformer model proposed in the text?


 QUESTION What is the main feature of the Transformer model proposed in the text?




 ANSWER: The main feature of the Transformer model is that it relies entirely on self-attention to compute representations of its input and output, without using sequence-aligned RNNs or convolution.
